In [0]:
from collections import defaultdict
import numpy as np
from pyspark.ml.linalg import SparseVector
from pyspark.sql.functions import explode
from pyspark import SparkFiles
from pyspark.sql import Row

import bz2
import json
import time
from pyspark.ml import Pipeline
from pyspark.ml.feature import * # CountVectorizer, Tokenizer, RegexTokenizer, HashingTF
from pyspark.ml.regression import * # RandomForestRegressor, LinearRegression, DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
def timeit(method):
    '''
    Decorator to time functions.
    '''
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()

        print('%r took %2.2f sec\n' % (method.__name__, te-ts))
              
        return result
    return timed

In [0]:
import re
# DATAFILE_PATTERN = '^(.+),"(.+)",(.*),(.*),(.*)'
ID_PATTERN = '"id":(.*?(?=,|}))'
UPS_PATTERN = '"ups":(.*?(?=,|}))'
BODY_PATTERN = '"body":(.*?(?=,|}))'
# DOWNS_PATTERN = '"downs":(.*?(?=,|}))'
SCORE_PATTERN = '"score":(.*?(?=,|}))'
# CONTROVERSIALITY_PATTERN = '"controversiality":(.*?(?=,|}))'

def removeQuotes(s):
    """ Remove quotation marks from an input string
    Args:
        s (str): input string that might have the quote "" characters
    Returns:
        str: a string without the quote characters
    """
    return ''.join(i for i in s if i!='"')

def parseDatafileLine(datafileLine):
    """ Parse a line of the data file using the specified regular expression pattern
    Args:
        datafileLine (str): input string that is a line from the data file
    Returns:
        tuple: a tuple including the parsed results using the given regular expression and without the quote characters
    """
    id_match = re.search(ID_PATTERN, datafileLine.decode('utf-8'))
    ups_match = re.search(UPS_PATTERN, datafileLine.decode('utf-8'))
    body_match = re.search(BODY_PATTERN, datafileLine.decode('utf-8'))
    score_match = re.search(SCORE_PATTERN, datafileLine.decode('utf-8'))
    
    if (id_match is None) or (ups_match is None) or (body_match is None) or (score_match is None):
        print('Invalid datafile line: %s' % datafileLine)
        return (datafileLine, -1)
    else:
        viralness = 0
        if int(score_match.group(1)) < -10 or int(score_match.group(1)) > 10:
            viralness = 1
        comment = (id_match.group(1), int(ups_match.group(1)), removeQuotes(body_match.group(1)), int(score_match.group(1)), viralness)
        return (comment, 1)

In [0]:
import sys
import os
from pyspark import SparkFiles

RC_PATH = '/FileStore/shared_uploads/ddk1@andrew.cmu.edu/RC_2007_10'

def parseData(path):
    """ Parse a data file
    Args:
        filename (str): input file name of the data file
    Returns:
        RDD: a RDD of parsed lines
    """
#     sc.addFile(path)
    return (sc
            .textFile(SparkFiles.get(path), 4, 0)
            .map(parseDatafileLine)
            .cache())

def loadData(path):
    """ Load a data file
    Args:
        path (str): input file name of the data file
    Returns:
        RDD: a RDD of parsed valid lines
    """

    raw = parseData(path).cache()
    
    failed = (raw
              .filter(lambda s: s[1] == -1)
              .map(lambda s: s[0]))
    for line in failed.take(10):
        print('%s - Invalid datafile line: %s' % (path, line))
    
    deleted = (raw
             .filter(lambda s: s[0][2] == '[deleted]')
             .map(lambda s: s[0]))
    
    valid = (raw
             .filter(lambda s: s[1] == 1)
             .filter(lambda s: s[0][2] != '[deleted]')
             .map(lambda s: s[0])
             .cache())
    viral = (raw
             .filter(lambda s: s[1] == 1)
             .filter(lambda s: s[0][4] == 1)
             .map(lambda s: s[0])
             .cache())
    nonviral = (raw
               .filter(lambda s: s[1] == 1)
               .filter(lambda s: s[0][4] == 0)
               .map(lambda s: s[0])
               .cache())
    nonviral_cut = nonviral.sample(False, viral.count()/nonviral.count())
    viral_nonviral_cut = viral.union(nonviral_cut)
    print('%s - Read %d lines, successfully parsed %d lines, failed to parse %d lines, %d lines were deleted, %d lines were viral, %d lines were non-viral, %d viral and non-viral lines were returned' % (path,
                                                                                                                                                                                                 raw.count(),
                                                                                                                                                                                                 valid.count(),
                                                                                                                                                                                                 failed.count(),
                                                                                                                                                                                                 deleted.count(),
                                                                                                                                                                                                 viral.count(),
                                                                                                                                                                                                 nonviral.count(),
                                                                                                                                                                                                 viral_nonviral_cut.count()))
    return viral_nonviral_cut

reddit = loadData(RC_PATH)

/FileStore/shared_uploads/ddk1@andrew.cmu.edu/RC_2007_10 - Read 150429 lines, successfully parsed 126320 lines, failed to parse 0 lines, 24109 lines were deleted, 13183 lines were viral, 137246 lines were non-viral, 26211 viral and non-viral lines were returned

In [0]:
sentenceDF = reddit.toDF().selectExpr("_1 as id", "_2 as ups", "_3 as body", "_4 as score", "_5 as viralness")
sentenceDF.show(n=5)

+---------+---+--------------------+-----+---------+
 id|ups| body|score|viralness|
+---------+---+--------------------+-----+---------+
"c0299b0"| 12|Is anyone else's ...| 12| 1|
"c0299b5"| 12|can't see beta.re...| 12| 1|
"c0299bv"| 22|This new comment ...| 22| 1|
"c0299ey"|157|There aren't many...| 157| 1|
"c0299fr"| 20|Shouldn't you hav...| 20| 1|
+---------+---+--------------------+-----+---------+
only showing top 5 rows

In [0]:
split_regex = r'\W+'
linebreak_regex = r'\\r\\n\\r\\n'

def simpleTokenize(string):
    """ A simple implementation of input string tokenization
    Args:
        string (str): input string
    Returns:
        list: a list of tokens
    """
    linebreak_removed_string = re.sub(linebreak_regex, " ", string)
    return list(filter(None, re.split(split_regex, linebreak_removed_string.lower())))

In [0]:
stopfile = "https://raw.githubusercontent.com/10605/data/master/hw1/stopwords.txt"
sc.addFile(stopfile)
stopwords = set(sc.textFile("file://" + SparkFiles.get("stopwords.txt")).collect())
print('These are the stopwords: %s' % stopwords)

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    return list(filter(lambda word: word not in stopwords,simpleTokenize(string)))

These are the stopwords: {'out', 'we', 'was', 'how', 'myself', 'for', 'they', 'about', 'then', 'both', 'so', 'don', 'as', 'any', 'after', 'you', 'why', 'been', 'where', 'by', 'yourself', 'a', 'did', 'their', 'doing', 'be', 'further', 'ours', 'now', 'am', 'her', 'yourselves', 'that', 'what', 'my', 'to', 'not', 'own', 'there', 'this', 'each', 'all', 'more', 'me', 'which', 'himself', 'nor', 'other', 'who', 'same', 'at', 'such', 't', 'up', 'than', 'can', 'too', 'these', 'while', 'before', 'ourselves', 'he', 'i', 'our', 'its', 'but', 'with', 'because', 'those', 'the', 'it', 'hers', 'just', 'over', 'between', 'had', 'does', 'have', 'and', 'some', 'or', 'only', 'when', 'below', 'in', 'if', 'theirs', 'again', 'his', 'whom', 'above', 'should', 'itself', 'themselves', 'until', 'are', 'she', 'will', 'from', 'into', 'no', 'your', 'few', 'herself', 'of', 'has', 'down', 'were', 'once', 'having', 'them', 'under', 'him', 'do', 'on', 'an', 'yours', 'being', 'off', 'very', 'through', 'most', 'against', 'here', 'is', 's', 'during'}

In [0]:
redditRecToToken = reddit.map(lambda line: (line[0], line[1], tokenize(line[2]),line[3], line[4]))

print(redditRecToToken.take(5))

def countTokens(vendorRDD):
    """ Count and return the number of tokens
    Args:
        vendorRDD (RDD of (recordId, tokenizedValue)): Pair tuple of record ID to tokenized output
    Returns:
        count: count of all tokens
    """
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    recordCount = vendorRDD.map(lambda line: len(line[0]))
    recordSum = recordCount.sum()
    return recordSum

totalTokens = countTokens(redditRecToToken)
print('There are %s tokens in the combined datasets' % totalTokens)

[('"c0299b0"', 12, ['anyone', 'else', 'recommended', 'page', 'completely', 'empty', 'mine'], 12, 1), ('"c0299b5"', 12, ['see', 'beta', 'reddit', 'com'], 12, 1), ('"c0299bv"', 22, ['new', 'comment', 'system', 'going', 'wonders', 'fibonacci', 'sequence', 'thread', 'http', 'reddit', 'com', 'info', '2mg72', 'comments', 'c2zuy0'], 22, 1), ('"c0299ey"', 157, ['aren', 'many', 'changes'], 157, 1), ('"c0299fr"', 20, ['shouldn', 'tested', 'changes', 'releasing', 'production', 'feels', 'like', 'huge', 'cock', 'release', 'else', 'would', 'explain', '48', 'hour', 'downtime', 'upgrade', 'edit', 'fixed', 'typo'], 20, 1)]
There are 235899 tokens in the combined datasets

In [0]:
def timeit(method):
    '''
    Decorator to time functions.
    '''
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()

        print('%r took %2.2f sec\n' % (method.__name__, te-ts))
              
        return result
    return timed

In [0]:
@timeit
def term_frequency(df, inputCol, outputCol, hashFeatures=None):
    '''
    Returns a DataFrame object containing a new row with the extracted features. 
    Passing hashed=True will return a Featured Hashed matrix.
    
    @params:
        df - DataFrame
        inputCol - name of input column from DataFrame to find features
        outputCol - name of the column to save the features
        hashFeatures - number of features for HashingTF, if None will perform 
            CountVectorization
    '''
    
    # since the number of features was not passed perform standard CountVectorization
    if hashFeatures is None:
        cv = CountVectorizer(inputCol=inputCol, outputCol=outputCol)
        feature_extractor = cv.fit(df)
    # otherwise perform a feature extractor with 
    else:
        feature_extractor = HashingTF(\
                              inputCol=inputCol, outputCol=outputCol, numFeatures=hashFeatures)
    
    # create a new DataFrame using either feature extraction method
    return feature_extractor.transform(df)

In [0]:
wordsFilteredDF = spark.createDataFrame(redditRecToToken).toDF("id", "ups", "filtered_words", "score", "viralness")

# Feature Hash the comment content
# number of features for Feature Hash matrix, reccomended too use power of 2
hashDF = term_frequency(\
    df=wordsFilteredDF, inputCol="filtered_words", outputCol="features", hashFeatures=1024)

# Display snippet of new DataFrame
hashDF.select('filtered_words','features').show(5)

'term_frequency' took 0.39 sec

+--------------------+--------------------+
 filtered_words| features|
+--------------------+--------------------+
[anyone, else, re...|(1024,[67,81,106,...|
[see, beta, reddi...|(1024,[346,541,55...|
[new, comment, sy...|(1024,[92,329,407...|
[aren, many, chan...|(1024,[308,637,75...|
[shouldn, tested,...|(1024,[49,80,127,...|
+--------------------+--------------------+
only showing top 5 rows

In [0]:
@timeit
def random_forest_regression(df, featuresCol, labelCol):
    '''
    Returns a DataFrame containing a column of predicted values of the labelCol.
    Predict the output of labelCol using values in featuresCol y = rf(x).
    
    @params:
        df - DataFrame
        featuresCol - input features, x
        labelCol - output variable, y
    '''
    # split the training and test data using the holdout method
    (trainingData, testData) = df.randomSplit([0.8, 0.2])
    
    # create the random forest regressor, limit number of trees to ten
    dtr = RandomForestRegressor(\
       featuresCol=featuresCol, labelCol=labelCol)
    
    # fit the training data to the regressor to create the model
    model = dtr.fit(trainingData)
    
    # create a DataFrame contained a column with predicted values of the labelCol
    predictions = model.transform(testData)
    
    return predictions

In [0]:
# train random forest regression
rfPredictions = random_forest_regression(df=hashDF,featuresCol="features",labelCol="score")

# compute the error
evaluator = RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfPredictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

'random_forest_regression' took 18.41 sec

Root Mean Squared Error (RMSE) on test data = 24.8089

In [0]:
rfPredictions.show(10)

+---------+---+--------------------+-----+---------+--------------------+------------------+
 id|ups| filtered_words|score|viralness| features| prediction|
+---------+---+--------------------+-----+---------+--------------------+------------------+
"c0299b0"| 12|[anyone, else, re...| 12| 1|(1024,[67,81,106,...| 12.58771816992639|
"c0299bv"| 22|[new, comment, sy...| 22| 1|(1024,[92,329,407...|12.316584194677791|
"c0299ey"|157|[aren, many, chan...| 157| 1|(1024,[308,637,75...| 12.58771816992639|
"c0299gj"| 18|[great, see, comm...| 18| 1|(1024,[25,92,179,...|12.780034319084425|
"c0299gw"| 16|[doesn, work, ll,...| 16| 1|(1024,[371,374,55...| 12.58771816992639|
"c0299h2"|-19| [deleted]| -19| 1| (1024,[1018],[1.0])| 4.043190445466253|
"c0299h5"|103|[discovered, outs...| 103| 1|(1024,[123,272,42...|12.780034319084425|
"c0299iv"| 55| [anonymously]| 55| 1| (1024,[73],[1.0])| 12.58771816992639|
"c0299jh"| 15| [hit, refresh]| 15| 1|(1024,[284,790],[...| 12.58771816992639|
"c0299kw"|-29|[think, need, hir...| -29| 1|(1024,[165,217,28...| 12.58771816992639|
+---------+---+--------------------+-----+---------+--------------------+------------------+
only showing top 10 rows

In [0]:
# train random forest regression
viralPredictions = random_forest_regression(df=hashDF,featuresCol="features",labelCol="viralness")

# compute the error
evaluator = RegressionEvaluator(labelCol="viralness", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(viralPredictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

'random_forest_regression' took 17.60 sec

Root Mean Squared Error (RMSE) on test data = 0.494195

In [0]:
viralPredictions.show(10)

+---------+---+--------------------+-----+---------+--------------------+------------------+
 id|ups| filtered_words|score|viralness| features| prediction|
+---------+---+--------------------+-----+---------+--------------------+------------------+
"c0299fx"| 15|[lots, zero, poin...| 15| 1|(1024,[71,93,330,...|0.5174605093929077|
"c0299hy"| 23|[whoa, n, nand, m...| 23| 1|(1024,[282,420,42...|0.5542663769279199|
"c0299i8"| 44|[trolled, reddit,...| 44| 1|(1024,[421,556,67...|0.5289265843808844|
"c0299il"| 25|[law, mandates, s...| 25| 1|(1024,[192,238,42...|0.5174605093929077|
"c0299iv"| 55| [anonymously]| 55| 1| (1024,[73],[1.0])|0.5174605093929077|
"c0299l5"| 20|[would, like, say...| 20| 1|(1024,[61,214,215...|0.5708458045572689|
"c0299lj"| 13| [rtfa]| 13| 1| (1024,[125],[1.0])|0.5174605093929077|
"c0299mb"| 85|[reddit, didn, no...| 85| 1|(1024,[556,597,92...| 0.565433982157996|
"c0299mz"| 35|[tip, delete, red...| 35| 1|(1024,[112,287,55...|0.5289265843808844|
"c0299nm"| 13| [m, complaining]| 13| 1|(1024,[19,282],[1...|0.5174605093929077|
+---------+---+--------------------+-----+---------+--------------------+------------------+
only showing top 10 rows

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
@timeit
def logistic_regression(df, featuresCol, labelCol, viralness):
    '''
    Returns a DataFrame containing a column of predicted values of the labelCol.
    Predict the output of labelCol using values in featuresCol y = rf(x).
    
    @params:
        df - DataFrame
        featuresCol - input features, x
        labelCol - output variable, y
    '''
    # split the training and test data using the holdout method
    print(df.count())
    if not viralness:
      df = df.filter((df.score >=0) & (df.score <10))
    print(df.show(10))
    print(df.count())
    (trainingData, testData) = df.randomSplit([0.8, 0.2])
    
    
    # TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
    # Given hyperparameters
    standardization = False
    elastic_net_param = 0.8
    reg_param = .3
    max_iter = 10

    lr = (LogisticRegression(featuresCol=featuresCol, labelCol=labelCol, regParam = reg_param, standardization = standardization, maxIter = max_iter,elasticNetParam = elastic_net_param))
#     lr = (LogisticRegressionWithLBFGS(featuresCol=featuresCol, labelCol=labelCol, regParam = reg_param, standardization = standardization, maxIter = max_iter,elasticNetParam = elastic_net_param))
#     model = LogisticRegressionWithLBFGS.train(trainingData, numClasses=5)
#     print(lr.numClasses)
#     
    lr_model_basic = lr.fit(trainingData)
#     lr_model_basic =LogisticRegressionWithLBFGS.train(trainingData.rdd(),numClasses=5)

    # YOUR CODE HERE
    # raise NotImplementedError()

    #print('intercept: {0}'.format(lr_model_basic.interceptVector))
#     print('length of coefficients: {0}'.format(len(lr_model_basic.coefficientMatrix)))
#     sorted_coefficients = sorted(lr_model_basic.coefficients)[:5]
    trainingSummary = lr_model_basic.summary
    accuracy = trainingSummary.accuracy
    print(accuracy)
    # create a DataFrame contained a column with predicted values of the labelCol
    predictions = lr_model_basic.transform(testData)
    
    return predictions

In [0]:
# train random forest regression
lrPredictions = logistic_regression(df=hashDF,featuresCol="features",labelCol="score", viralness=False)

# compute the error
evaluator = RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(lrPredictions)
print ("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

26211
+---------+---+--------------------+-----+---------+--------------------+
 id|ups| filtered_words|score|viralness| features|
+---------+---+--------------------+-----+---------+--------------------+
"c0299ar"| 3| [oh]| 3| 0| (1024,[776],[1.0])|
"c0299at"| 3| [like, one, time]| 3| 0|(1024,[319,386,68...|
"c0299aw"| 2|[also, wonder, di...| 2| 0|(1024,[243,704,98...|
"c0299bo"| 1| [woot, nice]| 1| 0|(1024,[435,842],[...|
"c0299bz"| 1| [m, testing, _]| 1| 0|(1024,[11,273,282...|
"c0299c4"| 3| [likes]| 3| 0| (1024,[372],[1.0])|
"c0299ca"| 3|[think, parent, l...| 3| 0|(1024,[492,522,67...|
"c0299ch"| 1|[d, rather, dead,...| 1| 0|(1024,[11,243,285...|
"c0299co"| 1|[would, happen, l...| 1| 0|(1024,[119,329,33...|
"c0299d0"| 1| [deleted]| 1| 0| (1024,[1018],[1.0])|
+---------+---+--------------------+-----+---------+--------------------+
only showing top 10 rows

None
11644
0.47289318303811056
'logistic_regression' took 18.34 sec

Root Mean Squared Error (RMSE) on test data = 2.28253

In [0]:
lrPredictions.show(10)

+---------+---+--------------------+-----+---------+--------------------+--------------------+--------------------+----------+
 id|ups| filtered_words|score|viralness| features| rawPrediction| probability|prediction|
+---------+---+--------------------+-----+---------+--------------------+--------------------+--------------------+----------+
"c0299ar"| 3| [oh]| 3| 0| (1024,[776],[1.0])|[0.57041738429234...|[0.09878433657725...| 1.0|
"c0299bz"| 1| [m, testing, _]| 1| 0|(1024,[11,273,282...|[0.57041738429234...|[0.09878433657725...| 1.0|
"c0299ca"| 3|[think, parent, l...| 3| 0|(1024,[492,522,67...|[0.57041738429234...|[0.09878433657725...| 1.0|
"c0299f1"| 1| [strange, comments]| 1| 0|(1024,[241,729],[...|[0.57041738429234...|[0.09878433657725...| 1.0|
"c0299fu"| 1| [deleted]| 1| 0| (1024,[1018],[1.0])|[0.57041738429234...|[0.09878433657725...| 1.0|
"c0299g8"| 1| [deleted]| 1| 0| (1024,[1018],[1.0])|[0.57041738429234...|[0.09878433657725...| 1.0|
"c0299nt"| 9|[usability, studi...| 9| 0|(1024,[119,120,20...|[0.57041738429234...|[0.09878433657725...| 1.0|
"c0299r5"| 1| [deleted]| 1| 0| (1024,[1018],[1.0])|[0.57041738429234...|[0.09878433657725...| 1.0|
"c0299uz"| 2|[like, remembers,...| 2| 0|(1024,[64,74,225,...|[0.57041738429234...|[0.09878433657725...| 1.0|
"c0299v6"| 1|[pictures, gray, ...| 1| 0|(1024,[819,891,97...|[0.57041738429234...|[0.09878433657725...| 1.0|
+---------+---+--------------------+-----+---------+--------------------+--------------------+--------------------+----------+
only showing top 10 rows

In [0]:
# train random forest regression
lrViralPredictions = logistic_regression(df=hashDF,featuresCol="features",labelCol="viralness", viralness=True)

# compute the error
evaluator = RegressionEvaluator(labelCol="viralness", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(lrViralPredictions)
print ("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

26211
+---------+---+--------------------+-----+---------+--------------------+
 id|ups| filtered_words|score|viralness| features|
+---------+---+--------------------+-----+---------+--------------------+
"c0299b0"| 12|[anyone, else, re...| 12| 1|(1024,[67,81,106,...|
"c0299b5"| 12|[see, beta, reddi...| 12| 1|(1024,[346,541,55...|
"c0299bv"| 22|[new, comment, sy...| 22| 1|(1024,[92,329,407...|
"c0299ey"|157|[aren, many, chan...| 157| 1|(1024,[308,637,75...|
"c0299fr"| 20|[shouldn, tested,...| 20| 1|(1024,[49,80,127,...|
"c0299fx"| 15|[lots, zero, poin...| 15| 1|(1024,[71,93,330,...|
"c0299gj"| 18|[great, see, comm...| 18| 1|(1024,[25,92,179,...|
"c0299go"| 77|[kept, hitting, r...| 77| 1|(1024,[71,183,912...|
"c0299gs"| 73|[subreddits, gett...| 73| 1|(1024,[176,178,23...|
"c0299gw"| 16|[doesn, work, ll,...| 16| 1|(1024,[371,374,55...|
+---------+---+--------------------+-----+---------+--------------------+
only showing top 10 rows

None
26211
0.5019600344201166
'logistic_regression' took 6.98 sec

Root Mean Squared Error (RMSE) on test data = 0.702214

In [0]:
lrViralPredictions.show(10)

+---------+---+--------------------+-----+---------+--------------------+--------------------+--------------------+----------+
 id|ups| filtered_words|score|viralness| features| rawPrediction| probability|prediction|
+---------+---+--------------------+-----+---------+--------------------+--------------------+--------------------+----------+
"c0299gw"| 16|[doesn, work, ll,...| 16| 1|(1024,[371,374,55...|[-0.0078401778404...|[0.49803996557988...| 1.0|
"c0299hd"| 27| [lurked, reddit]| 27| 1|(1024,[556,621],[...|[-0.0078401778404...|[0.49803996557988...| 1.0|
"c0299i8"| 44|[trolled, reddit,...| 44| 1|(1024,[421,556,67...|[-0.0078401778404...|[0.49803996557988...| 1.0|
"c0299ib"| 37| [mind]| 37| 1| (1024,[704],[1.0])|[-0.0078401778404...|[0.49803996557988...| 1.0|
"c0299im"| 11|[even, trouble, g...| 11| 1|(1024,[93,100,122...|[-0.0078401778404...|[0.49803996557988...| 1.0|
"c0299iv"| 55| [anonymously]| 55| 1| (1024,[73],[1.0])|[-0.0078401778404...|[0.49803996557988...| 1.0|
"c0299kw"|-29|[think, need, hir...| -29| 1|(1024,[165,217,28...|[-0.0078401778404...|[0.49803996557988...| 1.0|
"c0299l5"| 20|[would, like, say...| 20| 1|(1024,[61,214,215...|[-0.0078401778404...|[0.49803996557988...| 1.0|
"c0299lj"| 13| [rtfa]| 13| 1| (1024,[125],[1.0])|[-0.0078401778404...|[0.49803996557988...| 1.0|
"c0299lk"| 67|[reading, first, ...| 67| 1|(1024,[231,398,41...|[-0.0078401778404...|[0.49803996557988...| 1.0|
+---------+---+--------------------+-----+---------+--------------------+--------------------+--------------------+----------+
only showing top 10 rows